In [ ]:
import nest_asyncio
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from pyngrok import ngrok
import mlflow
import mlflow.sklearn
import joblib
import numpy as np
import uvicorn
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Guarda los vectorizadores
joblib.dump(vectorizer_todo, "vectorizer_todo.pkl")
joblib.dump(vectorizer_bajo, "vectorizer_bajo.pkl")
joblib.dump(vectorizer_alto, "vectorizer_alto.pkl")

In [ ]:
# Configurar MLFlow para registrar modelos
mlflow.set_tracking_uri("sqlite:///mlruns.db")

# Partiendo de los modelos entrenados: lda_todos, lda_altos, lda_bajos
# Se registran en MLFlow
with mlflow.start_run(run_name="lda_todos"):
    mlflow.sklearn.log_model(lda_todos, "model", registered_model_name="lda_todos")

with mlflow.start_run(run_name="lda_altos"):
    mlflow.sklearn.log_model(lda_altos, "model", registered_model_name="lda_altos")

with mlflow.start_run(run_name="lda_bajos"):
    mlflow.sklearn.log_model(lda_bajos, "model", registered_model_name="lda_bajos")

print("Modelos registrados en MLFlow")

In [ ]:
# Configuración para Colab
nest_asyncio.apply()

# Configurar URI de MLFlow
mlflow.set_tracking_uri("sqlite:///mlruns.db")

# Cargar los vectorizadores desde los archivos
vectorizer_todo_2 = joblib.load("vectorizer_todo.pkl")
vectorizer_bajo_2 = joblib.load("vectorizer_bajo.pkl")
vectorizer_alto_2 = joblib.load("vectorizer_alto.pkl")


# Inicializar FastAPI
app = FastAPI()

# Estructura para la solicitud
class TopicRequest(BaseModel):
    texto: str
    modelo: str  # Puede ser "todos", "altos", "bajos"

@app.post("/predict_topic/")
def predict_topic(request: TopicRequest):
    """
    API que predice el tópico dominante usando un modelo de LDA seleccionado desde MLFlow.
    """
    # Seleccionar el modelo y vectorizador
    if request.modelo == "todos":
        model_name = "lda_todos"
        vectorizer = vectorizer_todo_2
    elif request.modelo == "altos":
        model_name = "lda_altos"
        vectorizer = vectorizer_alto_2
    elif request.modelo == "bajos":
        model_name = "lda_bajos"
        vectorizer = vectorizer_bajo_2
    else:
        raise HTTPException(status_code=400, detail="El modelo debe ser 'todos', 'altos' o 'bajos'.")

    # Cargar el modelo desde MLFlow
    try:
        model_uri = f"models:/{model_name}/latest"
        lda_model = mlflow.sklearn.load_model(model_uri)
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"No se pudo cargar el modelo {model_name}: {str(e)}")

    # Transformar el texto en Bag of Words
    features = vectorizer.transform([request.texto])

    # Verificar si hay términos válidos
    if features.nnz == 0:
        raise HTTPException(status_code=400, detail="El texto no contiene términos reconocidos por el modelo.")

    # Predicción del tópico
    topic_distribution = lda_model.transform(features)
    dominant_topic = np.argmax(topic_distribution)

    # Respuesta
    return {
        "modelo": request.modelo,
        "tópico_dominante": int(dominant_topic),
        "distribución_tópicos": topic_distribution.tolist()
    }

# Inicia el túnel con ngrok
public_url = ngrok.connect(8000)
print(f"Tu API está disponible en: {public_url}")

# Inicia el servidor FastAPI
uvicorn.run(app, host="0.0.0.0", port=8000)